In [ ]:
import os
import time
from datetime import datetime

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager  # ✅ New import

import gspread
from google.oauth2.service_account import Credentials

# === CONFIGURATION ===

# Path to service account JSON
SERVICE_ACCOUNT_FILE = r"xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx.json"

# Google Sheet info
SPREADSHEET_KEY = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
WORKSHEET_NAME = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

# NGX URL
URL = "https://ngxgroup.com/exchange/data/equities-price-list/"

# Directories for data and logs
BASE_DIR = os.getcwd()
LOG_DIR = os.path.join(BASE_DIR, "logs")
DATA_DIR = os.path.join(BASE_DIR, "data")
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)

# Filenames
today_str = datetime.now().strftime("%Y-%m-%d")
CSV_FILE = os.path.join(DATA_DIR, f"data_{today_str}.csv")
LOG_FILE = os.path.join(LOG_DIR, "web_scrap_log.txt")


# === Logging helper ===
def log_message(message: str):
    """Append timestamped messages to log file using UTF-8 encoding"""
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"{datetime.now():%Y-%m-%d %H:%M:%S} - {message}\n")


# === Handle Cookie Consent ===
def handle_cookie_consent(driver, wait):
    try:
        cookie_button = wait.until(
            EC.element_to_be_clickable((By.ID, "cookie_action_close_header"))
        )
        driver.execute_script("arguments[0].click();", cookie_button)
        log_message("✅ Closed cookie popup")
    except Exception as e:
        log_message(f"ℹ️ Cookie popup not found or error: {e}")


# === Main Scraper Function ===
def scrape_and_push():
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--incognito")

    # ✅ Use ChromeDriverManager instead of a fixed chromedriver.exe path
    driver = webdriver.Chrome(
        service=webdriver.chrome.service.Service(ChromeDriverManager().install()),
        options=options,
    )
    wait = WebDriverWait(driver, 20)

    try:
        driver.get(URL)
        handle_cookie_consent(driver, wait)
        time.sleep(2)

        # Expand table to show all rows
        try:
            filter_option = wait.until(EC.element_to_be_clickable((
                By.XPATH, "//*[@id='latestdiclosuresEquities_length']/label/select/option[last()]"
            )))
            filter_option.click()
            time.sleep(2)
            log_message("✅ Table expanded to show all rows")
        except Exception:
            log_message("⚠️ Could not expand table rows")

        # Scrape table
        table = wait.until(EC.presence_of_element_located((By.ID, "latestdiclosuresEquities")))
        table_html = table.get_attribute("outerHTML")
        soup = BeautifulSoup(table_html, "html.parser")

        headers = [th.get_text(strip=True) for th in soup.find("thead").find_all("th")]
        rows = []
        for row in soup.find("tbody").find_all("tr"):
            cells = row.find_all("td")
            rows.append([cell.get_text(strip=True) for cell in cells])

        if headers and rows:
            df = pd.DataFrame(rows, columns=headers)

            # Clean company names
            if "Company" in df.columns:
                df["Company"] = df["Company"].str.strip()

            df["Date"] = today_str
            df.to_csv(CSV_FILE, index=False, encoding="utf-8")
            log_message(f"✅ Data saved locally ({len(df)} rows)")

            # === Push to Google Sheets ===
            try:
                scope = [
                    "https://spreadsheets.google.com/feeds",
                    "https://www.googleapis.com/auth/spreadsheets",
                    "https://www.googleapis.com/auth/drive.file",
                    "https://www.googleapis.com/auth/drive",
                ]
                creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=scope)
                client = gspread.authorize(creds)

                spreadsheet = client.open_by_key(SPREADSHEET_KEY)
                sheet = spreadsheet.worksheet(WORKSHEET_NAME)

                # Clear existing data
                sheet.clear()

                # Update headers and data
                sheet.update(values=[df.columns.tolist()], range_name="A1")
                sheet.update(values=df.values.tolist(), range_name="A2")

                log_message(f"✅ NGXUPDATE sheet successfully updated with {len(df)} rows")
                print(f"✅ NGXUPDATE sheet successfully updated with {len(df)} rows")

            except Exception as e:
                log_message(f"❌ Google Sheets update failed: {e}")
                print(f"❌ Google Sheets update failed: {e}")
        else:
            log_message("⚠️ No data found in table")
            print("⚠️ No data found in table")

    except Exception as e:
        log_message(f"❌ Scraping failed: {e}")
        print(f"❌ Scraping failed: {e}")
    finally:
        driver.quit()
        log_message("Browser closed.")


# === MAIN ===
if __name__ == "__main__":
    with open(LOG_FILE, "w", encoding="utf-8") as f:
        f.write(f"{today_str} - Log started\n")
    scrape_and_push()